In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME      PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
yadin-ca  GCE       4                                       RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan 11 19:07 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'ir-project-z'
full_path = f"gs://{bucket_name}/"
paths=[]
client = storage.Client()
bucket = client.bucket(bucket_name)
blobs = bucket.list_blobs(prefix='multiwiki/')
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)
paths = paths[1:]

***GCP setup is complete!***

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [9]:
parquetFile = spark.read.parquet(*paths)
doc_title_pairs = parquetFile.select("title", "id").rdd


We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [10]:
# Count number of wiki pages
parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [11]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_title_gcp.py

inverted_index_title_gcp.py


In [12]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_title_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [13]:
from inverted_index_title_gcp import *

A few notes: 
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab. 

In [14]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings, dir):
# function that partitions the posting lists into buckets, writes out all posting lists in a bucket to disk,
# and returns the posting locations for each bucket.
  partition_postings = postings.map(lambda x: (token2bucket_id(x[0]),x)).groupByKey()
  return partition_postings.map(lambda x: InvertedIndex.write_a_posting_list(x, dir))

In [15]:
def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  # YOUR CODE HERE
  end_production = []
  checklist = []
  for token in tokens:
    if(token not in all_stopwords and token not in checklist):
      checklist.append(token)
      end_production.append((token,(id,tokens.count(token))))
  return end_production
# -------------------------------------------------------------------------------
def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  sorted_by_first = sorted(unsorted_pl, key=lambda tup: tup[0])
  return sorted_by_first
# -------------------------------------------------------------------------------
def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  return postings.map(lambda tup: (tup[0],len(tup[1])))
# -------------------------------------------------------------------------------
def partition_postings_and_write(postings, dir):
# function that partitions the posting lists into buckets, writes out all posting lists in a bucket to disk,
# and returns the posting locations for each bucket.
  partition_postings = postings.map(lambda x: (token2bucket_id(x[0]),x)).groupByKey()
  return partition_postings.map(lambda x: InvertedIndex.write_a_posting_list(x, dir))
# -------------------------------------------------------------------------------

In [17]:
# word counts map
word_counts_title = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings_title = word_counts_title.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df --title dont need filter
# postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df_title = calculate_df(postings_title)
w2df_dict_title = w2df_title.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings_title,bucket_name).collect()


In [18]:
# collect all posting lists locations into one super-set
super_posting_locs_title = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp_title'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs_title[k].extend(v)

Putting it all together

In [19]:
# Create inverted index instance
inverted = InvertedIndex()
inverted.doc_name = parquetFile.select("id","title").rdd.collectAsMap()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs_title
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict_title
# write the global stats out
inverted.write_index('.', 'index_title')
# upload to gs
index_src = "index_title.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp_title/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://index_title.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

\ [1 files][236.5 MiB/236.5 MiB]                                                
Operation completed over 1 objects/236.5 MiB.                                    


In [20]:
!gsutil ls -lh $index_dst

236.47 MiB  2023-01-12T16:30:41Z  gs://ir-project-z/postings_gcp_title/index_title.pkl
TOTAL: 1 objects, 247960065 bytes (236.47 MiB)
